<a href="https://colab.research.google.com/github/luisam19/course_optimizacion/blob/main/AnalisisSensibilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='056938'> **Importar librerias** </font>

In [ ]:
!pip install -q amplpy
from amplpy import AMPL, tools
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import sympy as sp
from sympy.matrices import Matrix
from sympy.vector import Vector
import numpy as np

# <font color='056938'> **Descripción genérica de un PL** </font>

Para la discusión de la sensibilidad consideraremos una descripción genérica del programa lineal como sigue:

> Debemos decidir el nivel de algunas actividades $𝑥_𝑖$ que maximice el beneficio obtenido, considerando un beneficio $𝑐_𝑖$ asociado a cada actividad y haciendo uso de una cantidad limitada de recursos $𝑏_𝑗$, de los cuales cada unidad de de actividad $𝑖$ usa una cantidad $𝑎_{𝑖𝑗}$

Lo cuál matemáticamente representamos como:

\begin{align*}
\text{maximizar} \quad &c_1 x_1 + c_2 x_2 + \ldots + c_n x_n\\
    &\text{s.a.}\\
    &a_{11} x_1 + a_{12} x_2 + \ldots + a_{1n} x_n \leq b_1 \\
    &\quad \quad \quad \vdots \\
    & a_{m1} x_1 + a_{m2} x_2 + \ldots + a_{mn} x_n \leq b_m \\
    &x_1 \geq 0, \quad x_2 \geq 0, \quad \ldots, \quad x_n \geq 0
\end{align*}

De forma similar, para el problema de minimización tendriamos, el problema lo expresaremos asi:

> Debemos decidir el nivel de algunas actividades $𝑥_𝑖$ que minimice el costo generado, considerando un costo $𝑐_𝑖$ asociado a cada actividad con el fin de satisfacer una demanda esperada de un bien o recurso $𝑏_𝑗$, para los cuales cada unidad de de actividad $𝑖$ aporta una cantidad $𝑎_{𝑖𝑗}$


\begin{align*}
\text{minimizar} \quad &c_1 x_1 + c_2 x_2 + \ldots + c_n x_n\\
    &\text{s.a.}\\
    &a_{11} x_1 + a_{12} x_2 + \ldots + a_{1n} x_n \geq b_1 \\
    &\quad \quad \quad \vdots \\
    & a_{m1} x_1 + a_{m2} x_2 + \ldots + a_{mn} x_n \geq b_m \\
    &x_1 \geq 0, \quad x_2 \geq 0, \quad \ldots, \quad x_n \geq 0
\end{align*}

# <font color='056938'> **Problema base** </font>

Para la discusión de esta unidad usaremos como base un problema de mezcla

## <font color='8EC044'> **Descripción del problema** </font>

Una empresa produce cuatro tipos de productos: Producto A, Producto B, Producto C y Producto D. Cada producto es una mezcla de diferentes ingredientes, y la empresa desea maximizar su beneficio. El beneficio por unidad de cada producto es el siguiente:

| Producto | Bneficioe |
|-------------|---------------------|
| A           | 20        |
| B           | 15        |
| C           | 25        |
| D           | 50        |

El proceso de producción requiere los siguientes ingredientes por unidad de cada producto:

| Producto | Ingrediente 1 | Ingrediente 2 |Ingrediente 3 |
|----------|---------------|---------------|---------------|
| A        | 2             | 3             | 1             |
| B        | 1             | 2             | 2             |
| C        | 3             | 1             | 4             |
| D        | 2             | 4             | 3             |

La empresa cuenta con las siguientes cantidades de ingredientes disponibles:


| Ingrediente | Cantidad disponible |
|-------------|---------------------|
| 1           | 60 unidades        |
| 2           | 50 unidades        |
| 3           | 100 unidades        |


Además, la empresa quiere asegurarse de que el número de puntos por producción sostenible sea mayor al menos de 50 unidades. Donde los puntos asociados a cada unidad de producto son:

| Producto | puntos |
|-------------|---------------------|
| A           | 4        |
| B           | 3        |
| C           | 2        |
| D           | 1        |



## <font color='8EC044'> **Formulación del problema** </font>


\begin{align*}
\text{maximizar} z = &20x_A + 15x_B + 25x_C + 50x_D\\
\text{s.a.} \\
\text{Ingredientes 1:} & \quad 2x_A + x_B + 3x_C + 2x_D \leq 60 \\
\text{Ingredientes 2:} & \quad 3x_A + 2x_B + x_C + 4x_D \leq 50 \\
\text{Ingredientes 3:} & \quad x_A + 2x_B + 4x_C + 3x_D \leq 100 \\
\text{Producción sost.:} & \quad 4x_A + 3x_B + 2x_C + 1x_D \geq 50 \\
\text{No negatividad:} & \quad x_A \geq 0, \, x_B \geq 0, \, x_C \geq 0, \, x_D \geq 0
\end{align*}

## <font color='8EC044'> **Implementación** </font>

En este caso usaremos `AMPL` para modelar el problema y `CPLEX` para resolverlo, puesto que de esta forma podemos obtener fácilmente la información que requeriremos en el analisis de sensibilidad. Dependiendo del modelador y del solver de solución puede ser posible o no obtener esta infromación fácilmente.

In [ ]:
ampl = tools.ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # Los optimizadores que vamos a usar
    license_uuid="default") # license to use (Aqui hay que poner su licencia :-;

En nuestro caso, crearemos la  función `create_model()` que genera el modelo para usarla en caso que tengamos que  generarlo para distintos conjuntos de datos.

In [ ]:
def create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min):
  #Conjuntos
  ampl.eval(
      """ reset;
          set Products;          #Conjunto de producos
          set Ingredients;				#Conjunto de ingredientes
          #Parameters
          param benefit{Products}>=0;              # Beneficio de cada producto
          param dispon{Ingredients}>=0;              # disponibilidad de cada ingrediente
          param requerim {Ingredients,Products}>=0; # Requerimiento de ingrediente por producto
          param volumn{Products}>=0;              # Aporte por unidad de producto
          param prod_min >=0;                             # Numero minimo de productos

          #Variables
          var x{j in Products}>=0;  # Cantidad de producto a fabricar

          #Objective function
          maximize TotalBenefit: sum{j in Products}benefit[j]*x[j];

          #Constraints
          subject to Disponibilidad{i in Ingredients}:
          sum{j in Products}requerim[i,j]*x[j]<=dispon[i];

          subject to ProdMin:
          sum{j in Products}volumn[j]*x[j]>=prod_min;""")

  # instantiate the model
  #Parametros cargados directamente
  ampl.set["Products"] = products
  ampl.set["Ingredients"] = ingredients
  ampl.param["benefit"] = benefits
  ampl.param["dispon"] = disponibility
  ampl.param["requerim"] = requirement
  ampl.param["volumn"] = volumn
  ampl.param["prod_min"] = prod_min

  return ampl


Construimos los datos de la instancia para alimentar la función que crea el modelo

In [ ]:
products = ["A", "B", "C", "D"]
ingredients = [1, 2, 3] #En Python contamos desde 0 ! Acuerdate de esto
benefits = {"A": 20, "B": 15, "C": 25, "D": 50}
disponibility = {1: 60, 2: 50, 3: 100}
requirement = {
    (1, 'A'): 2,
    (1, 'B'): 1,
    (1, 'C'): 3,
    (1, 'D'): 2,
    (2, 'A'): 3,
    (2, 'B'): 2,
    (2, 'C'): 1,
    (2, 'D'): 4,
    (3, 'A'): 1,
    (3, 'B'): 2,
    (3, 'C'): 4,
    (3, 'D'): 3
}
volumn = {"A": 4, "B": 3, "C": 2, "D": 1}
prod_min = 50



Para efectos de correr el modelo y poder evidenciar los concpetos del analisis de sensibilidad usaremos `cplex` desactivando las opciones de presolver y activando el componente de sensibilidad

In [ ]:
%%ampl_eval
option solver cplex;
option cplex_options 'sensitivity';
option presolve 0;

Corremos el modelo e imprimimos la solución de las variables de decisión y el valor de la función objetivo

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
# Imprimimos los resultados
TotalBenefit = ampl.get_objective("TotalBenefit")
print("\nObjective is:", TotalBenefit.get().value())
ampl.get_data("x").to_pandas()

# <font color='056938'> **Análisis de sensibilidad a cambios en los parámetros del modelo** </font>

El programa lineal asume que los valores de todos los parámetros son deterministicos. Sin embargo, conscientes de que esto en la un caso real puede no ser siempre cierto, es importante considerar el impacto que tiene en la solución el cambio en los valores de dichos parámetros.

## <font color='8EC044'> **Relajar y ajustar restricciones** </font>

En primer lugar es importante comprender que sucede cuando relajamos una restricción o la hacemos más ajustada. Como se presenta en el siguiente gráfico.

![](https://docs.google.com/uc?export=download&id=13CdKyVOhZTej8Q6rCWgq7QsBK6OrRLdg)


De manera general podemos concluir que,

> Al **relajar** una restricción en un programa lineal el valor óptimo de la función objetivo solo puede mejorar o por lo menos permanecer igual

En caso contrario,

> Al **ajustar** una restricción en un programa lineal el valor óptimo de la función objetivo solo puede empeorar o cuando más permanecer igual

Debemos preguntarnos ahora como puede relajarse o ajustarce una restricción de un progrema lineal. Veamos algunas formas en que esto sucede

### <font color='46B8A9'> **Cambios en el lado derecho de las restricciones** </font>  

Cambios en el lado derecho que relajan la una restricción cuando incrementan o por lo menos conservan igual el número de soluciones que la satisfacen. Siendo así, podriamos decir que

> Un **incremeto** del valor del lado derecho de una restriccón de *$\leq$* o una **reducción** en el lado derecho de una restriccón de $\geq$ **relajan** la restriccón

En el problema base que estamos considerando

\begin{align*}
\text{maximizar} z = &20x_A + 15x_B + 25x_C + 50x_D\\
&\text{s.a.} \\
 & \quad 2x_A + x_B + 3x_C + 2x_D \leq 60 & \text{(1)}\\
& \quad 3x_A + 2x_B + x_C + 4x_D \leq 50 & \text{(2)}\\
& \quad x_A + 2x_B + 4x_C + 3x_D \leq 100 & \text{(3)}\\
 & \quad 4x_A + 3x_B + 2x_C + 1x_D \geq 50 & \text{(4)}\\
\text{No negatividad:} & \quad x_A \geq 0, \, x_B \geq 0, \, x_C \geq 0, \, x_D \geq 0
\end{align*}

Las restricciones (1), (2), y (3) se relajan incrementando el lado derecho, mientras que la restricción (4) se relaja reduciendo el lado derecho


Veamos por ejemplo, como cambia la función objetivo respecto a cambios en las restricciones de disponibilidad de recursos

In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
values = []
change_param = ampl.get_parameter("dispon")
# iterate over different values of the parameter
for value in range(0, 200):
  change_param.set_values({3: value})
  ampl.solve()
  status = ampl.get_value('solve_result_num')
  if status < 99:
    values.append((value, TotalBenefit.get().value()))

df = pd.DataFrame(values, columns=["param_value", "obj_function"])

# Create line plot using Plotly Express
fig = px.line(df, x='param_value', y='obj_function', title='Changes in RHS parameter')
# Update axes labels
fig.update_xaxes(title_text='RHS')
fig.update_yaxes(title_text='Obj function')

# Show plot
fig.show()

Hagamos lo mismo respecto a la restricción de (4)

In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
values = []
change_param = ampl.get_parameter("prod_min")
print(change_param)
# iterate over different values of the parameter
for value in range(0, 200):
  change_param.set_values([value])
  ampl.solve()
  status = ampl.get_value('solve_result_num')
  if status < 99:
    values.append((value, TotalBenefit.get().value()))

df = pd.DataFrame(values, columns=["param_value", "obj_function"])

# Create line plot using Plotly Express
fig = px.line(df, x='param_value', y='obj_function', title='Changes in RHS parameter')
# Update axes labels
fig.update_xaxes(title_text='RHS')
fig.update_yaxes(title_text='Obj function')

# Show plot
fig.show()

Es importante notar que ante cambios en el vector $b$ la solución óptima actual puede dejar de ser factible. Por ello, estariamos interesados en determinar cuál es el intervalo en el cual podría variar cada uno de los valores de $b$

Debemos encontrar el rango de valores en el que debe encontrase el nuevo valor del lado derecho, $b_k^{'} = b_{k} +Δb_k, Para ello consderamos el sistema de desigualdades.

$x_B = B^-{1}\begin{bmatrix}
b_{1}  \\
b_{k} +Δb_k \\
...  \\
b_{m} \\
\end{bmatrix} \geq 0$

Consideremos el rango de valores en los que debe encontrarse el coeficiente $b_2$ del lado derecho de la segunda restricción

In [ ]:
d= sp.symbols('d')
B = Matrix(( [1, 3, 2, 0], [2, 1, 4, 0], [2, 4, 3, 1], [3, 2, 1, 0]))
b = Matrix([60, 50+d, 100, 50])
Binv = B.inv(method="LU")
Binv*b

Lo que permite concluir que $\dfrac{-160}{7}\leq Δb_2 \leq 70$, es decir:
> La solución conformada por las variables básicas actuales continuará siendo óptima mientras que $\dfrac{190}{7}\leq b_2 \leq 120$

In [ ]:
d= sp.symbols('d')
cB = Matrix([15, 25, 50, 0])
cNB = Matrix([20])
B = Matrix(( [1, 3, 2, 0], [2, 1, 4, 0], [2, 4, 3, 1], [3, 2, 1, 0]))
aj = Matrix([2, 3, 1, 4+d])

Binv = B.inv(method="LU")
cR = cNB - cB.transpose()*B.inv()*aj
cR


#### <font color='46B8A9'> **Ejercicio** </font>

Determine el rango de valores en los cuáles debe encontrarse el valor de $b_1$ para que la solución conformada por las variables básicas actuales continue siendo óptima

Dependiendo del optimizador y el lenguaje de modelación empleado, la información respecto al rango de los valores en lo que pueden variar los coeficientes del lado derecho puede obtenerse automáticamente. En el caso particular de `ampl` la información puede obtenerse asi:

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
# Obtenemos la  información de las restricciones
ampl.display('_conname, _con.slack, _con.down,  _con.current, _con.up')



### <font color='46B8A9'> **Cambios en los coeficientes de las restricciones** </font>  

Los cambios en los coeficientes de las restricciones pueden afectar no solo la óptimalidad de la solución sino afectar también su factibilidad. De forma general, podría decirse que,

> Qué al **disminuir** alguno de los coeficientes en una restricción de **$\leq$** o al **incrementar** algun coeficiente en una restricción de $\geq$ el problema es **relajado**

Similarmente,

> Qué al **incrementar** alguno de los coeficientes en una restricción de **$\leq$** o al **disminuir** algun coeficiente en una restricción de $\geq$ el problema es más **ajustado**



In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
values = []
change_param = ampl.get_parameter("requerim") # identify the parameter to change
# iterate over different values of the parameter
for value in np.arange(0, 10.1, 0.1):
  change_param.set_values({(3, 'C'): value}) #(2, 'A')
  ampl.solve()
  status = ampl.get_value('solve_result_num')
  if status < 99:
    values.append((value, TotalBenefit.get().value()))

df = pd.DataFrame(values, columns=["param_value", "obj_function"])

# Create line plot using Plotly Express
fig = px.line(df, x='param_value', y='obj_function', title='Changes in RHS parameter')
# Update axes labels
fig.update_xaxes(title_text='RHS')
fig.update_yaxes(title_text='Obj function')

# Show plot
fig.show()

#### <font color='46B8A9'> **Ejercicio** </font>
Experimente cambiando el valor de distintos coeficientes de las restricciones? Es igual el comportamiento en todos los casos?

Es posible notar que el efecto depende de si el  coeficiente que cambia esta asociado a una variable  básica o una variable no básica.

> Al cambiar el coeficiente asociado a una **variable no básica** esta puede volverse atractiva para ser ingresada a la base y aunque la solución óptima actual no dejaria de ser factible, si podría dejar de ser óptima

Búsquemos los valores para los cuales un cambio en el aporte que el producto 1 hace a la  restricción (4) hace que la solución deje de ser óptima (el producto $x_A$ sea considerado en la mezcla de producción).

> $(4+\Delta a_{41})x_A + 3x_B + 2x_C + 1x_D \geq 50$

Para lo cual resolvemos  la siguiente desigualdad:

$\overline{c}_{A}  = 20 + [15, 25, 50, 0] \begin{bmatrix}
1& 3& 2& 0  \\
2& 1& 4& 0\\
2& 4& 3& 1 \\
3& 2& 1& 0 \\
\end{bmatrix}^{-1}\begin{bmatrix}
2  \\
3 \\
1  \\
4 +Δb_4 \\
\end{bmatrix}>0$




In [ ]:
d= sp.symbols('d')
cB = Matrix([15, 25, 50, 0])
cNB = Matrix([20])
B = Matrix(( [1, 3, 2, 0], [2, 1, 4, 0], [2, 4, 3, 1], [3, 2, 1, 0]))
aj = Matrix([2, 3, 1, 4+d])

Binv = B.inv(method="LU")
cR = cNB - cB.transpose()*B.inv()*aj
cR

Para que la variable $x_A$ sea atractiva su costo reducido debera ser mayor que 0. Es decir, buscamos los valores para los que

$4\Delta a_{41} - \dfrac{42}{5} > 0\implies \Delta a_{41} > \dfrac{21}{10} $

Es decir $a_{41}$ debe ser mayor que 6.1 para que el producto $A$ sea atractivo para ingresar a la base.

Veamos el efecto en el cambio en dicho parámetro

In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
change_param = ampl.get_parameter("volumn")
change_param.set_values({"A": 6.1})
ampl.solve()
# Imprimimos los resultados
TotalBenefit = ampl.get_objective("TotalBenefit")
print("\nObjective is:", TotalBenefit.get().value())
ampl.get_data("x").to_pandas()

Ahora, en el caso en que cambia el coeficiente de una variable no básica este análisis es usualmente mucho más complejo y dificilmente puede encontrarse análiticamente <font color='46B8A9'> **¿Por qué?** </font>

### <font color='46B8A9'> **Adicionar o eliminar restricciones** </font>  

Adicionar o eliminar restricciones cambia la región factible del programa lineal por lo que puede afectar no solo la optimalidad de las restricciones sino también su factibilidad. en geberal

> Un programa lineal se **realaja al eliminar restricciones** y se hace más **ajustado al incluir nuevas restricciones**

## <font color='8EC044'> **Cambios en los coeficientes de la función objetivo** </font>

Los cambios en la función objetivo ni relajan ni ajustan la región factible. Sin embargo, pueden tener efectos importantes en la optimalidad de la solución actual. Es posible que un cojunto de variables diferente genere una solución mejor al cambiar alguno de dichos coeficientes. Note, por ejemplo, cómo cambia la función objetivo a medida que cambia el beneficio del producto $A$ en la función objetivo

In [ ]:
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
values = []
change_param = ampl.get_parameter("benefit")
# iterate over different values of the parameter
for value in range(0, 100):
  change_param.set_values({'A': value})
  ampl.solve()
  status = ampl.get_value('solve_result_num')
  if status < 99:
    values.append((value, TotalBenefit.get().value()))

df = pd.DataFrame(values, columns=["param_value", "obj_function"])

# Create line plot using Plotly Express
fig = px.line(df, x='param_value', y='obj_function', title='Changes in RHS parameter')
# Update axes labels
fig.update_xaxes(title_text='Coeficiente función objetivo')
fig.update_yaxes(title_text='Obj function')

# Show plot
fig.show()

#### <font color='46B8A9'> **Ejercicio** </font>
Use el siguiente script para determinar el valor debería tomar la utilidad generada por el producto 4 de modo que este sea atractivo para considerar en al mezcla de productos

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
change_param = ampl.get_parameter("benefit")
change_param.set_values({'A': 20})
ampl.solve()
# Imprimimos los resultados
TotalBenefit = ampl.get_objective("TotalBenefit")
print("\nObjective is:", TotalBenefit.get().value())
ampl.get_data("x").to_pandas()

Note que es posible obtener esta información de forma analítica sin tener que recurrir a prueba y error de diferentes valores. Para ello considere que si se hacen cambios en los coeficientes de la función objetivo, solo podría afectarse la optimalidad lo cual se válida con los costos reducidos.

$\overline{c}_{NB}^T  = c_{NB}^T-c_B^T B^{-1}N$

> En un problema de maximización la solución seguirá siendo óptima mientras que los costos reducidos continien siendo $\leq 0$ y dejará de ser óptima en caso contrario

similarmente,

> En un problema de minimización la solución seguirá siendo óptima mientras que los costos reducidos continuen siendo $\geq 0$ y dejará de ser óptima en caso contrario


Identifiquemos el valor del beneficio $c_A$ asociado al producto A a partir del cuál la solución deja de ser óptima. Para ello resolvemos el siguiente sistema

$\overline{c}_{A}  = (20+\Delta c_A) + [15, 25, 50, 0] \begin{bmatrix}
1& 3& 2& 0  \\
2& 1& 4& 0\\
2& 4& 3& 1 \\
3& 2& 1& 0 \\
\end{bmatrix}^{-1}\begin{bmatrix}
2  \\
3 \\
1  \\
4 \\
\end{bmatrix} >0$


In [ ]:
d= sp.symbols('d')
cB = Matrix([15, 25, 50, 0])
cNB = Matrix([20+d])
B = Matrix(( [1, 3, 2, 0], [2, 1, 4, 0], [2, 4, 3, 1], [3, 2, 1, 0]))
aj = Matrix([2, 3, 1, 4])

Binv = B.inv(method="LU")
cR = cNB - cB.transpose()*B.inv()*aj
cR

Buscamos los valores para los que

$\Delta c_{A}  > \dfrac{42}{5} $

Es decir $c_{A}$ debe ser mayor que $20 + \dfrac{42}{5}$ para que el producto $A$ sea atractivo para ingresar a la base.

#### <font color='46B8A9'> **Ejercicio** </font>
Encuentre los valores para los que cambiado el beneficio $c_3$ del producto $C$ la solución deja de ser óptima

Dependiendo del optimizador y el lenguaje de modelación empleado, la información respecto al rango de los valores en lo que pueden variar los coeficientes de la función objetivo puede obtenerse automáticamente. En el caso particular de `ampl` la información puede obtenerse asi:

In [ ]:
# Creamos el mdoelo y resolvemos el modelo
ampl = create_model(ampl, products, ingredients, benefits, disponibility, requirement, volumn, prod_min)
ampl.solve()
# Obtenemos la  información de las restricciones
ampl.display('_var.current, _var.down,_var.current,_var.up')



# <font color='056938'> **Práctica** </font>

La compañia **Nutricol** esta interesado en desrrollar una app que permita al usuario crear una dieta balanceada con base en el lstado de posibles ingredientes y requerimientos nutricionales específicos. Para ello ha contratado nuestros servicios para desrrollar un modelo que encuentre una dieta diaria de mínimo costo



## <font color='8EC044'> **Verbalización del modelo** </font>
Consideraremos una versión bastante simplificada del problema. Para ello definamos los elementos



#### <font color='46B8A9'> **Conjuntos** </font>

$I$: conjunto de ingredientes

$N$ conjunto de nutrientes

Note que el conjunto de ingredientes podria a su ve subdividirse en tipos de ingredientes como carnes, verduras, granos y frutas.

#### <font color='46B8A9'> **Parámetros** </font>

Debemos cosiderar los siguientes parametros

* costo por kilo de cada ingreiente
* aporte nutricional (en cada nutriente) de cada ingrediente
* catindad mínima de cada tipo de nutriente
* cantidad máxima de cada tipo de nutriente
* Debe existir proporcionalidad entre cada tipo de nutrientes (ej, la carne no debe excer más del 30% del menú)

#### <font color='46B8A9'> **Decisiones** </font>
Debemos decidir la cantidad de cada alimento que debe incluirse en el menú diario

#### <font color='46B8A9'> **Restricciones** </font>
La decisión esta sujeta a que:
* se cumplan los requisitos mínimos de cada nutriente
* se cumplan los requisitos máximos de cada nutriente
* No se excedan las cantidades máximas por tipos de nutrientes


#### <font color='46B8A9'> **Objetivo** </font>
Minimizar el costo total del menú


| Variable | Descripción                    |
|----------|--------------------------------|
| $I$      | Conjunto de ingredientes       |
| $S$      | Conjunto de tipos de ingredientes  |
| $N$      | Conjunto de nutrientes         |


| Parámetro | Descripción |
|-----------|-------------|
| $c_i$     | Costo por kilogramo del ingrediente $i \in I$. |
| $n_{ij}$  | Aporte nutricional del ingrediente $i \in I$ al nutriente $j \in N$. |
| $amin_j$   | Cantidad mínima requerida del nutriente $j \in N$. |
| $amax_j$   | Cantidad máxima permitida del nutriente $j \in N$. |
| $p_{k}$  | Proporción máxima de tipo de ingrediente $k$ en la mezcla. |
| $s_{ik}$  | Indicador binario que define si el ingrediente $i$ es del tipo $k$ |

La variable de decisión
> $x_i$: Cantidad de kilogramos del ingrediente $i \in I$ a incluir en el menú diario.




## <font color='8EC044'> **Formulación del modelo** </font>


\begin{align}
  \text{Minimizar} \quad \sum_{i \in I} c_i x_i\\
  \text{s.a.}\\
    \sum_{i \in I} n_{ij} x_i &\geq \text{amin}_j & \forall j \in N \\
    \sum_{i \in I} n_{ij} x_i &\leq \text{amax}_j & \forall j \in N \\
    \frac{\sum_{i \in I} s_{ik}x_i}{\sum_{i \in I} x_i} &\leq p_{ij} & \forall k \in S, \forall i \in I\\
    x_i &\geq 0 \quad \forall i \in I
\end{align}


## <font color='8EC044'> **Implementación del modelo** </font>

Creamos el objeto `ampl`

In [ ]:
ampl = tools.ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # Los optimizadores que vamos a usar
    license_uuid="default") # license to use (Aqui hay que poner su licencia :-;


Definimos una función que cree el modelo

In [ ]:
def create_model(ampl, ingredients, types_ing, nutrients, cost, contrib, min_nutri, max_nutri, pmix, ind_type):
  #Conjuntos
  ampl.eval(
    """ reset;
    set I;   # Conjunto de ingredientes
    set S;   # Conjunto de tipos de ingredientes
    set N;   # Conjunto de nutrientes

    param cost{i in I};     # Costo por kilogramo del ingrediente i
    param contrib{i in I, j in N};  # Aporte nutricional del ingrediente i al nutriente j
    param min_nutri{j in N};  # Cantidad mínima requerida del nutriente j
    param max_nutri{j in N};  # Cantidad máxima permitida del nutriente j
    param pmix{k in S};     # Proporción máxima de tipo de ingrediente k en la mezcla
    param ind_type{i in I, k in S};  # Indicador binario que define si el ingrediente i es del tipo k

    var x{i in I} >= 0;  # Cantidad de kilogramos del ingrediente i a incluir en el menú diario

    minimize Total_cost:
        sum{i in I} cost[i] * x[i];

    subject to min_nutrient{j in N}:
        sum{i in I} contrib[i, j] * x[i] >= min_nutri[j];

    subject to max_nutrient{j in N}:
        sum{i in I} contrib[i, j] * x[i] <= max_nutri[j];

    subject to mix_proportion{k in S}:
        sum{i in I} ind_type[i, k] * x[i] - sum{i in I} x[i]*pmix[k] <= 0;""")

  # instantiate the model
  ampl.set["I"] = ingredients
  ampl.set["S"] = types_ing
  ampl.set["N"] = nutrients
  ampl.param["cost"] = cost
  ampl.param["contrib"] = contrib
  ampl.param["min_nutri"] = min_nutri
  ampl.param["max_nutri"] = max_nutri
  ampl.param["pmix"] = pmix
  ampl.param["ind_type"] = ind_type

  return ampl

Consideramos la instancia de datos del modelo

In [ ]:
# Datos de ejemplo con nombres reales
ingredients = ["Pollo", "Res", "Pescado", "Tomate", "Brócoli", "Lechuga", "Arroz", "Frijol", "Lenteja", "Aguacate", "Mango", "Banano"]
types_ing = ["Carne", "Verduras", "Granos", "Frutas"]
nutrients = ['Proteína', 'Grasa', 'Carbohidratos', 'Fibra', 'Vitaminas']

# Costo por kilogramo del ingrediente por kilo (miles de pesos)
cost = {
    "Pollo": 15.6, "Res": 18.4, "Pescado": 19, "Tomate": 3.8, "Brócoli": 10.4, "Lechuga": 2.8,
    "Arroz": 5.1, "Frijol": 7.9, "Lenteja": 14.8, "Aguacate": 10.4, "Mango": 7.2, "Banano": 2.7
}

# Aporte nutricional del ingrediente al nutriente (en unidades por kg)
contrib = {
    ('Pollo', 'Proteína'): 25, ('Pollo', 'Grasa'): 10, ('Pollo', 'Carbohidratos'): 0, ('Pollo', 'Fibra'): 0, ('Pollo', 'Vitaminas'): 1,
    ('Res', 'Proteína'): 20, ('Res', 'Grasa'): 15, ('Res', 'Carbohidratos'): 0, ('Res', 'Fibra'): 0, ('Res', 'Vitaminas'): 1,
    ('Pescado', 'Proteína'): 22, ('Pescado', 'Grasa'): 8, ('Pescado', 'Carbohidratos'): 0, ('Pescado', 'Fibra'): 0, ('Pescado', 'Vitaminas'): 2,
    ('Tomate', 'Proteína'): 1, ('Tomate', 'Grasa'): 0, ('Tomate', 'Carbohidratos'): 5, ('Tomate', 'Fibra'): 2, ('Tomate', 'Vitaminas'): 15,
    ('Brócoli', 'Proteína'): 3, ('Brócoli', 'Grasa'): 0, ('Brócoli', 'Carbohidratos'): 5, ('Brócoli', 'Fibra'): 10, ('Brócoli', 'Vitaminas'): 20,
    ('Lechuga', 'Proteína'): 1, ('Lechuga', 'Grasa'): 0, ('Lechuga', 'Carbohidratos'): 2, ('Lechuga', 'Fibra'): 3, ('Lechuga', 'Vitaminas'): 10,
    ('Arroz', 'Proteína'): 6, ('Arroz', 'Grasa'): 1, ('Arroz', 'Carbohidratos'): 80, ('Arroz', 'Fibra'): 4, ('Arroz', 'Vitaminas'): 2,
    ('Frijol', 'Proteína'): 8, ('Frijol', 'Grasa'): 1, ('Frijol', 'Carbohidratos'): 60, ('Frijol', 'Fibra'): 8, ('Frijol', 'Vitaminas'): 3,
    ('Lenteja', 'Proteína'): 10, ('Lenteja', 'Grasa'): 1, ('Lenteja', 'Carbohidratos'): 50, ('Lenteja', 'Fibra'): 15, ('Lenteja', 'Vitaminas'): 4,
    ('Aguacate', 'Proteína'): 2, ('Aguacate', 'Grasa'): 15, ('Aguacate', 'Carbohidratos'): 8, ('Aguacate', 'Fibra'): 7, ('Aguacate', 'Vitaminas'): 10,
    ('Mango', 'Proteína'): 1, ('Mango', 'Grasa'): 0, ('Mango', 'Carbohidratos'): 20, ('Mango', 'Fibra'): 3, ('Mango', 'Vitaminas'): 30,
    ('Banano', 'Proteína'): 1, ('Banano', 'Grasa'): 0, ('Banano', 'Carbohidratos'): 30, ('Banano', 'Fibra'): 2, ('Banano', 'Vitaminas'): 20
}


# Cantidad mínima requerida del nutriente (en unidades)
min_nutri = {'Proteína': 50, 'Grasa': 30, 'Carbohidratos': 50, 'Fibra': 25, 'Vitaminas': 10}

# Cantidad máxima permitida del nutriente (en unidades)
max_nutri = {'Proteína': 150, 'Grasa': 50, 'Carbohidratos': 200, 'Fibra': 50, 'Vitaminas': 30}

# Proporción máxima de tipo de ingrediente en la mezcla
pmix = {"Carne": 0.4, "Verduras": 0.3, "Granos": 0.2, "Frutas": 0.5}

# Indicador binario que define si el ingrediente es del tipo k
ind_type = {(ing, ing_type):0 for ing in ingredients for ing_type in types_ing}
ind_type_sparce = {
    ("Pollo", "Carne"): 1, ("Res", "Carne"): 1, ("Pescado", "Carne"): 1,
    ("Tomate", "Verduras"): 1, ("Brócoli", "Verduras"): 1, ("Lechuga", "Verduras"): 1,
    ("Arroz", "Granos"): 1, ("Frijol", "Granos"): 1, ("Lenteja", "Granos"): 1,
    ("Aguacate", "Frutas"): 1, ("Mango", "Frutas"): 1, ("Banano", "Frutas"): 1}
ind_type.update(ind_type_sparce)



Definimos algunos parámetros para el optimizador

In [ ]:
%%ampl_eval
option solver cplex;
option cplex_options 'sensitivity';
option presolve 0;

Creamos el modelo, resolvemos e imprimimos la solución

In [ ]:
ampl = create_model(ampl, ingredients, types_ing, nutrients, cost, contrib, min_nutri, max_nutri, pmix, ind_type)
ampl.solve()
# Imprimimos los resultados
Total_cost = ampl.get_objective("Total_cost")
print("\nObjective is:", Total_cost.get().value())
ampl.get_data("x").to_pandas()

## <font color='8EC044'> **Análisis de sensibilidad** </font>

Desarrolle el analisis de sendibilidad respecto a los parámetros del modelo interpretando sus resultados de acuerdo al contexto del problema

In [ ]:
# Escriba aquí su respuesta

# <font color='056938'> **Referencias** </font>

RARDIN, Ronald L. Optimization in operations research. Upper Saddle River, NJ: Prentice Hall, 1998. Chapter 6